<img src="https://drive.google.com/uc?export=view&id=10rLzZzH5ctQ4bPRDZTbZk5tQMc9sDO0W" width="100%"></img>

# Cassandra y Pandas
---

En este notebook mostraremos cómo leer y escribir datos en _Cassandra_ desde `pandas`.

Primero instalaremos e importaremos las librerías Necesarias:

In [1]:
!pip install cassandra-driver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.8 MB 3.9 MB/s 


In [2]:
import pandas as pd
import numpy as np
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

Ahora, realizaremos la conexión a la base de datos de [astra](https://astra.datastax.com/) tal y cómo se mostró en el notebook `1_cassandra_python.ipynb`:

> **Nota**: recuerde subir los archivos `bundle.zip` y `token.csv`.

In [3]:
cloud_config= {
        "secure_connect_bundle": "bundle.zip" # nombre de su archivo de bundle
}
token = pd.read_csv("token.csv")

In [4]:
auth_provider = PlainTextAuthProvider(
        token["Client Id"].iloc[0],
        token["Client Secret"].iloc[0]
        )
cluster = Cluster(
        cloud=cloud_config,
        auth_provider=auth_provider
        )
session = cluster.connect()

ERROR:cassandra.connection:Closing connection <LibevConnection(140297155448336) 789432d4-2a70-4a5b-842a-0a7bed508af1-us-east1.db.astra.datastax.com:29042:475c4b73-d944-4e5c-8824-4adff0141231> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


## **1. Escritura**
---

Primero, vamos a crear un conjunto de datos sintético en `pandas`

In [5]:
df = pd.DataFrame(
        data=np.random.uniform(size=(500, 4)),
        columns=[f"columna{i}" for i in range(1, 5)]
        ).reset_index().rename(columns={"index": "id"})

In [6]:
df.head()

,id,columna1,columna2,columna3,columna4
0,0,0.547520,0.748064,0.611651,0.718892
1,1,0.388229,0.919191,0.911576,0.266947
2,2,0.933285,0.704215,0.326310,0.899866
3,3,0.962829,0.692830,0.384948,0.912019
4,4,0.643189,0.112101,0.553849,0.789617


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        500 non-null    int64  
 1   columna1  500 non-null    float64
 2   columna2  500 non-null    float64
 3   columna3  500 non-null    float64
 4   columna4  500 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 19.7 KB


Ahora, veamos cómo escribir esta tabla dentro de _Cassandra_, primero creamos la tabla, debe reemplazar `<nombre_keyspace>` por el nombre de su KEYSPACE.

In [8]:
query = """
CREATE TABLE
    test.tabla_pandas (
        id INT PRIMARY KEY,
        columna1 FLOAT,
        columna2 FLOAT,
        columna3 FLOAT,
        columna4 FLOAT
        )
;
"""
session.execute(query)

Ahora insertamos los valores desde `pandas`. Primero, preparamos los campos de la tabla que serán insertados:

In [9]:
query = """
INSERT INTO
    test.tabla_pandas (
            id, columna1, columna2,
            columna3, columna4
            )
VALUES
    (?, ?, ?, ?, ?)
;
"""
prepared = session.prepare(query)

Con esto, podemos insertar los valores desde `pandas`:

In [10]:
for row in df.itertuples():
    session.execute(
            prepared,
            (
                row.id, row.columna1, row.columna2,
                row.columna3, row.columna4
                )
            )

## **2. Lectura**
---

Ahora, veamos cómo podemos extraer una tabla de _Cassandra_ como un dataframe de `pandas`. Para ello usaremos la siguiente consulta:

In [11]:
query = """
SELECT
    id,
    columna1 + columna2 AS columna5,
    columna3 + columna4 AS columna6
FROM
    test.tabla_pandas
;
"""

In [12]:
df = pd.DataFrame(session.execute(query).all())
df.head()

,id,columna5,columna6
0,23,0.327734,0.817405
1,114,1.066705,0.557381
2,53,0.833028,0.441245
3,110,1.471545,1.435076
4,91,0.546575,1.387706


Finalmente, eliminamos la tabla creada para este ejercicio:

In [13]:
query = """
DROP TABLE test.tabla_pandas;
"""
session.execute(query, timeout=None)

## **3. Recursos Adicionales**
---

* [Python read cassandra data into pandas](https://stackoverflow.com/questions/41247345/python-read-cassandra-data-into-pandas).
* [How to insert pandas dataframe into cassandra(https://stackoverflow.com/questions/49108809/how-to-insert-pandas-dataframe-into-cassandra).

## **4. Créditos**
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Diseño, desarrollo del notebook y material audiovisual**

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*